In [19]:
from PIL import Image, ImageTk
import tkinter as tk
import cv2
import os
import numpy as np
from tensorflow.keras.models import model_from_json
import operator
import time
import sys, os
import matplotlib.pyplot as plt
import hunspell
from string import ascii_uppercase
import speech_recognition as sr
import pyttsx3 ##

phrases_list = ['Hello ','Male ','Female ','Yes ','No ','Where ','When ','What ','Email ','Name ','House ','Work ','Time ','Help ','I LOVE YOU ','Stand ','Family ','Marriage ','You ','We ','Nice ','Meet ','Owe ','Come ','Go ','Paint ']


class Application:
    def __init__(self):
        self.flag=0
#         self.directory = 'model'
        self.hs = hunspell.Hunspell("C:/Users/Admin/OneDrive/Documents/Desktop/en_US", "C:/Users/Admin/OneDrive/Documents/Desktop/en_US")
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        
        ##    
        self.speak_engine=pyttsx3.init()
        self.speak_engine.setProperty("rate",100)
        voices=self.speak_engine.getProperty("voices")
        self.speak_engine.setProperty("voice",voices[0].id)   
        ##
             
        self.json_file = open("C:/Users/Admin/Downloads/MODEL_WF-AZ_LNET_1400_newphrases.json", "r")
        self.model_json = self.json_file.read()
        self.json_file.close()
        self.loaded_model = model_from_json(self.model_json)
        self.loaded_model.load_weights("C:/Users/Admin/Downloads/MODEL_WF-AZ_LNET_1400_newphrases.h5")

        
        self.ct = {}
        self.ct['blank'] = 0
        self.blank_flag = 0
        for i in phrases_list:
            self.ct[i] = 0
        print("Loaded model from disk")
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1300x1300")
        self.panel = tk.Label(self.root)
        self.panel.place(x = -60, y = 50, width = 800, height = 400)   #panel big box image
        self.panel2 = tk.Label(self.root) # initialize image panel
        self.panel2.place(x = 350, y = 50, width = 310, height = 310) #panel 2 small gray box image
        
        self.T = tk.Label(self.root)
        self.T.place(x=500,y = 2)
        self.T.config(text = "Sign Language to Text",font=("Times New Roman",28,"bold"))
        
        self.panel3 = tk.Label(self.root) # Current SYmbol
        self.panel3.place(x = 400,y=470)                                          #640.......changeee
        self.T1 = tk.Label(self.root)
        self.T1.place(x = 10,y = 470)                                             #640.......changeeee
        self.T1.config(text="Character :",font=("Courier",20,"bold"))
        
        self.panel4 = tk.Label(self.root) # Word
        self.panel4.place(x = 165,y=510)                  
        self.T2 = tk.Label(self.root)
        self.T2.place(x = 10,y = 510)                     
        self.T2.config(text ="Word :",font=("Courier",20,"bold"))
        
        
        self.panel5 = tk.Label(self.root) # Sentence
        self.panel5.place(x = 180,y=550)       
        self.T3 = tk.Label(self.root)
        self.T3.place(x = 10,y =550)           
        self.T3.config(text ="Sentence :",font=("Courier",20,"bold"))
        
        ## dialogue formation testing
#         self.panel5_1 = tk.Label(self.root) # person 1
#         self.panel5_1.place(x = 350,y=550)       
        self.T3_1 = tk.Label(self.root)
        self.T3_1.place(x = 700,y =80)           
        self.T3_1.config(text ="person 1 :",font=("Courier",20,"bold"))
        
        
#         self.panel5_2 = tk.Label(self.root) # person 2
#         self.panel5_2.place(x = 450,y=200)       
        self.T3_2 = tk.Label(self.root)
        self.T3_2.place(x = 1125,y =210)           
        self.T3_2.config(text ="person 2", font=("Courier",20,"bold"))
        
        
        ###########

        self.T4 = tk.Label(self.root)
        self.T4.place(x = 250,y =580)  #820
        self.T4.config(text = "Suggestions",fg="red",font = ("Courier",20,"bold"))
        
        self.btcall = tk.Button(self.root,command = self.action_call,height = 0,width = 0)
        self.btcall.config(text = "About",font = ("Courier",14))
        self.btcall.place(x = 1200, y = 0)

      
        self.bt1=tk.Button(self.root, command=self.action1,height = 0,width = 0) #char
        self.bt1.place(x = 25,y=650)  #850
        
        self.bt2=tk.Button(self.root, command=self.action2,height = 0,width = 0) #word
        self.bt2.place(x = 150,y=650)
        
        self.bt3=tk.Button(self.root, command=self.action3,height = 0,width = 0) #sent
        self.bt3.place(x = 300,y=650)
        
        self.bt4=tk.Button(self.root, command=self.action4,height = 0,width = 0)
        self.bt4.place(x = 450,y=650)

        self.bt5=tk.Button(self.root, command=self.action5,height = 0,width = 0)
        self.bt5.place(x = 625,y=650)
        
        self.bt3_2=tk.Button(self.root, height = 5,width = 20, wraplength=150)   #user - listen
        self.bt3_2.place(x = 725,y=215)
        
        self.bt7=tk.Button(self.root, text="Start", command=self.start,font=("Courier",16))
        self.bt7.place(x = 575,y=450)
        
        self.bt8=tk.Button(self.root, text="Stop ", command=self.stop,font=("Courier",16))
        self.bt8.place(x = 575,y=495)
        
        
        

        ##
        self.clear = tk.Button(self.root)
        self.clear.place(x=1060, y=450)
        self.clear.config(text="Clear", font=("Courier", 20), wraplength=100, command=self.clear_fun)
        
        self.listen = tk.Button(self.root)
        self.listen.place(x=900, y=450)
        self.listen.config(text="Listen", font=("Courier", 20), wraplength=100, command=self.listen_fun)
        
        self.speak = tk.Button(self.root)
        self.speak.place(x=760, y=450)
        self.speak.config(text="Speak", font=("Courier", 20), wraplength=100, command=self.speak_fun)
        self.str = " "
        self.ccc=0
        self.word = " "
        self.current_symbol = ""
        self.photo = "Empty"


        self.word1= " "
        self.word2 = " "
        self.word3 = " "
        self.word4 = " "

        self.video_loop()
        
        ##

    def video_loop(self):
        ok, frame = self.vs.read()
        if ok:
            cv2image = cv2.flip(frame, 1)
            x1 = int(0.5*frame.shape[1])
            y1 = 10
            x2 = frame.shape[1]-10
            y2 = int(0.5*frame.shape[1])
            cv2.rectangle(frame, (x1-1, y1-1), (x2+1, y2+1), (255,0,0) ,1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)
            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)
            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)
            cv2image = cv2image[y1:y2, x1:x2]
            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray,(5,5),2)
            th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
            self.predict(res)
            self.current_image2 = Image.fromarray(res)
            imgtk = ImageTk.PhotoImage(image=self.current_image2)
            
            self.panel2.imgtk = imgtk
            self.panel2.config(image=imgtk)
            
            self.panel3.config(text=self.current_symbol,font=("Courier",20))    
            self.panel4.config(text=self.word,font=("Courier",20))
            self.panel5.config(text=self.str,font=("Courier",20))
            
            self.bt3_1=tk.Button(self.root, height = 2,width = 20)  #person 1 
            self.bt3_1.place(x = 850,y=80)
            self.bt3_1.config(text=self.str, font=("Courier", 20), wraplength=500)
            
            predicts=self.hs.suggest(self.word)
            if(len(predicts) > 0):
                self.bt1.config(text=predicts[0],font = ("Courier",20))
            else:
                self.bt1.config(text="")
                
            if(len(predicts) > 1):
                self.bt2.config(text=predicts[1],font = ("Courier",20))
            else:
                self.bt2.config(text="")
                
            if(len(predicts) > 2):
                self.bt3.config(text=predicts[2],font = ("Courier",20))
            else:
                self.bt3.config(text="")
                
            if(len(predicts) > 3):
                self.bt4.config(text=predicts[3],font = ("Courier",20))
            else:
                self.bt4.config(text="")
                
            if(len(predicts) > 4):
                self.bt5.config(text=predicts[4],font = ("Courier",20))
            else:
                self.bt5.config(text="") 
                
        self.root.after(30, self.video_loop)  #30 milisecs for real time 
        
        
    def predict(self,test_image):
        if self.flag==1:
            self.panel3.config(text="I work",font=("Courier",20))    
            
            test_image = cv2.resize(test_image, (128,128))
            result = self.loaded_model.predict(test_image.reshape(1, 128, 128, 1))
#        
            prediction={}
#             prediction['blank'] = result[0][0]
            inde = 0
            for i in phrases_list:
                prediction[i] = result[0][inde]
#                 if i==26:
#                     break
                inde += 1
        #LAYER 1
            prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
            self.current_symbol = prediction[0][0]
        

        #This code checks if the predicted symbol is 'blank', which indicates that no gesture was detected. If this is the case, it sets the count for each letter in ascii_uppercase to 0. If the predicted symbol is not 'blank', it increments the count for that symbol in the ct dictionary. The ct dictionary keeps track of the count of each symbol that has been predicted so far.
            if(self.current_symbol == 'blank'):
                for i in phrases_list:
                    self.ct[i] = 0             
            self.ct[self.current_symbol] += 1
        
            if(self.ct[self.current_symbol] > 30): #60000000000000000000000000000000000000000000000000000000000
                for i in phrases_list:
                    if i == self.current_symbol:
                        continue
                    tmp = self.ct[self.current_symbol] - self.ct[i]
                    if tmp < 0:
                        tmp *= -1
                    if tmp <= 20:
                        self.ct['blank'] = 0
                        for i in phrases_list:
                            self.ct[i] = 0
                        return
                self.ct['blank'] = 0
                for i in phrases_list:
                    self.ct[i] = 0
                if self.current_symbol == 'blank':
                    if self.blank_flag == 0:
                        self.blank_flag = 1
                        if len(self.str) > 0:
                            self.str += " "
                        self.str += self.word
                        self.word = ""
                else:
                    if(len(self.str) > 16):       #
                        self.str = ""
                    self.blank_flag = 0
                    self.word += self.current_symbol+ " "
    def action1(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 0):
            self.word=""
            self.str+=" "
            self.str+=predicts[0]
    def action2(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 1):
            self.word=""
            self.str+=" "
            self.str+=predicts[1]
    def action3(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 2):
            self.word=""
            self.str+=" "
            self.str+=predicts[2]
    def action4(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 3):
            self.word=""
            self.str+=" "
            self.str+=predicts[3]
    def action5(self):
    	predicts=self.hs.suggest(self.word)
    	if(len(predicts) > 4):
            self.word=""
            self.str+=" "
            self.str+=predicts[4]    
    
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
    
    def destructor1(self):
        print("Closing Application...")
        self.root1.destroy()
        
    def speak_fun(self):
        self.speak_engine.say(self.str)
        self.speak_engine.runAndWait()
        
    def start(self):
        self.flag=1
    
    def stop(self):
        self.flag=0
        self.current_symbol=""
        
    def clear_fun(self):
        self.str=" "
        self.word=" "
        self.word1 = " "
        self.word2 = " "
        self.word3 = " "
        self.word4 = " "
        self.dest=" "
        
    def listen_fun(self):
#         self.bt3_2.config(text="I started",font = ("Courier",20))
        r = sr.Recognizer()
        

        with sr.Microphone() as source:    #since we are taking online audio
            r.adjust_for_ambient_noise(source)  #to adjust ambient noise
#             self.bt3_2.config(text="I started",font = ("Courier",20))

# #             print("Please say something ")
#             self.bt3_2.config(text="Please say something",font = ("Courier",20))

            audio = r.listen(source)

# #             print("Reconizing Now ... ")
#             self.bt3_2.config(text="Processing",font = ("Courier",20))



            try:
                dest = r.recognize_google(audio)
#                 print("You have said : " + dest)
                self.bt3_2.config(text= dest,font = ("Courier",20))

#             web.get(path).open(dest)

            except Exception as e:
                print("Error : " + str(e))

    def action_call(self) :
        
        self.root1 = tk.Toplevel(self.root)
        self.root1.title("About")
        self.root1.protocol('WM_DELETE_WINDOW', self.destructor1)
        self.root1.geometry("900x900")
        
        # img = cv2.imread("Pictures/sir.jpg", 1)
        # # img = cv2.resize(img, (300, 300))
        # cv2.imwrite("Pictures/sir.png", img)
        # return 
        
        self.tx = tk.Label(self.root1)
        self.tx.place(x = 330,y = 20)
        self.tx.config(text = "Efforts By", fg="red", font = ("Courier",20,"bold"))

        self.photo1 = tk.PhotoImage(file="C:/Users/Admin/OneDrive/Pictures/Screenshots/2023-04-10.png")
        self.w1 = tk.Label(self.root1, image = self.photo1)
        self.w1.place(x = 20, y = 105)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 20,y = 250)
        self.tx6.config(text = "RC/nIIT2016141", font = ("Courier",15,"bold"))

        self.photo2 = tk.PhotoImage(file='Pictures/n.png')
        self.w2 = tk.Label(self.root1, image = self.photo2)
        self.w2.place(x = 200, y = 105)
        self.tx2 = tk.Label(self.root1)
        self.tx2.place(x = 200,y = 250)
        self.tx2.config(text = "Nitin/nIIT2016132", font = ("Courier",15,"bold"))

        
        self.photo3 = tk.PhotoImage(file='Pictures/l.png')
        self.w3 = tk.Label(self.root1, image = self.photo3)
        self.w3.place(x = 380, y = 105)
        self.tx3 = tk.Label(self.root1)
        self.tx3.place(x = 380,y = 250)
        self.tx3.config(text = "Luv/nIIT2016085", font = ("Courier",15,"bold"))

        self.photo4 = tk.PhotoImage(file='Pictures/sh.png')
        self.w4 = tk.Label(self.root1, image = self.photo4)
        self.w4.place(x = 560, y = 105)
        self.tx4 = tk.Label(self.root1)
        self.tx4.place(x = 560,y = 250)
        self.tx4.config(text = "Sheldon/nIIT2016137", font = ("Courier",15,"bold"))
        
        self.photo5 = tk.PhotoImage(file='Pictures/s.png')
        self.w5 = tk.Label(self.root1, image = self.photo5)
        self.w5.place(x = 740, y = 105)
        self.tx5 = tk.Label(self.root1)
        self.tx5.place(x = 740,y = 250)
        self.tx5.config(text = "Siddhant/nIIT2016069", font = ("Courier",15,"bold"))
        
        self.tx7 = tk.Label(self.root1)
        self.tx7.place(x = 170,y = 360)
        self.tx7.config(text = "Under the supervision of", fg="red", font = ("Courier",20,"bold"))

        self.photo6 = tk.PhotoImage(file='Pictures/sir.png')
        self.w6 = tk.Label(self.root1, image = self.photo6)
        self.w6.place(x = 350, y = 420)
        self.tx6 = tk.Label(self.root1)
        self.tx6.place(x = 230,y = 670)
        self.tx6.config(text = "Dr. Vrijendra Singh", font = ("Courier",20,"bold"))

print("Starting Application...")
pba = Application()
pba.root.mainloop()

Starting Application...
Loaded model from disk
Error : 
Closing Application...
